## Background exposure FIRST ESTIMATION for each volume of the detector between 2.4 - 2.5 MeV using Gonzalo's results on efficiency

We are going to use per isotope and volume the efficiency and activity to compute the number of events needed to simulate in nexus, in order to measure a minimum number of events in the detector. Also, we compute the exposure required for that.

In [1]:
import pandas as pd
import numpy as np

We read and compute the efficiences for each volume and isotope (214Bi, 208Tl, etc)

We have a file that per isotope and volume has:
* **nsim**: number of simulated events in nexus
* **nsaved**: number of events that were saved in the files (that is, detected)
* **npass**: number of events from those saved that passed the IC processing
* **nenergy**: number of events from those processed that had a total energy between 2.4-2.5MeV

So we compute the efficiency of having them (nsaved/nsim), and the efficiency of them also being in the Qbb window (nenergy/nsim)

In [2]:
#read file
efficiencies = pd.read_csv('efficiencies_2.4_2.5_MeV.csv')
#compute efficiencies
efficiencies['eff'] = efficiencies.nsaved / efficiencies.nsim
efficiencies['eff_Qbb'] = efficiencies.nenergy / efficiencies.nsim

efficiencies.drop(['nsim', 'nsaved', 'npass', 'nenergy'], axis = 1, inplace=True)

We read the activities now and set up the DF to merge with the efficiencies DF.

In [11]:
#read and rename activities file
activities = pd.read_excel('NEXT100_activities.ods', sheet_name=' DB  ACTIVITIES', engine='odf')
activities = activities.rename(columns = {'NEXUS VOLUME':'G4Volume', 'Bi-214':'214Bi', 'Co-60':'60Co', 'K-40':'40K', 'Tl-208':'208Tl'})

Bi_act = activities[['G4Volume', '214Bi']].copy()
Tl_act = activities[['G4Volume', '208Tl']].copy()
Bi_act['Isotope'] = '214Bi'
Tl_act['Isotope'] = '208Tl'

act = Bi_act.rename(columns={'214Bi':'act'}).append(Tl_act.rename(columns={'208Tl':'act'}))

#merge activities and efficiencies 
eff_and_act = efficiencies.merge(act, on = ['G4Volume', 'Isotope'], how = 'outer')


We add a column with the counts/year we want to have at the end of the processing (using the ones in the 2.4-2.5MeV region from Gonzalo's thesis)

In [14]:
#total_final_counts stands for the number of counts we want to have from one isotope
eff_and_act['total_final_counts'] = eff_and_act['Isotope'].apply(lambda x: {'208Tl':1800, 
                                                                            '214Bi':296, 
                                                                            '0nubb':0, 
                                                                            'muons':900, 
                                                                            '137Xe':33}[x])

In [15]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.12,296


At this point we have a df with the columns:
* **Isotope**: the name of the isotope
* **G4Volume**: the name of the volume
* **eff**: nexus efficiency for each isotope and volume
* **eff_Qbb**: efficiency for each isotope and volume in the energy window of 2.4-2.5MeV
* **act**: activity (mBq) for each isotope and volume
* **total_final_counts**: number of counts we want to have for a specific isotope, which are going to be distributed in each volume

We now will normalize for each isotope the efficiency x activity to compute the required counts per volume. For that I create the following function:

In [16]:

def distribute_final_counts(eff_and_act, eff = 'eff'):
    '''
    Function that distributes the desired counts per isotope into the different volumes attending to their
    efficiency and activity. It computes a normalized factor of efficiency times activity per volume that
    we can use to distribute the event counts in each volume. Finally the counts are rounded up.

    Variables:
        eff_and_act: DF
            DataFrame that contains the information per isotope and volume of their efficiencies and activities, and
            also the total desired counts for an isotope
        eff: STR
            Selects to compute the regular efficiency (nexus efficiency for each volume) or the energy efficiency (events 
            in the desired energy)
    '''
    #multiply the efficiency and the activity
    eff_and_act['eff_x_act'] = eff_and_act[eff] * eff_and_act['act']

    #sum all those factors to normalize
    sum_eff_x_act = pd.DataFrame(eff_and_act.groupby('Isotope')['eff_x_act'].sum()).reset_index().rename({'eff_x_act':'sum_eff_x_act'}, axis = 1)
    #merge and divide
    eff_and_act = eff_and_act.merge(sum_eff_x_act, on = 'Isotope', how = 'outer')
    eff_and_act['norm_eff_x_act'] = eff_and_act['eff_x_act'] / eff_and_act['sum_eff_x_act']

    #decide which efficiency to use
    if eff == 'eff_Qbb': final_name = 'final_counts_Qbb'
    else: final_name = 'final_counts'

    #distribute the counts according to the computed factor, round up the counts and clean the DF
    #if we apply ceil, the exposure changes for each volume and it shouldn't
    eff_and_act[final_name] = (eff_and_act['total_final_counts'] * eff_and_act['norm_eff_x_act'])#.apply(np.ceil)
    eff_and_act.drop(['eff_x_act', 'sum_eff_x_act', 'norm_eff_x_act'], axis = 1, inplace = True)
    return eff_and_act

We now apply the function, two times to do it with both efficiencies

In [17]:
eff_and_act = distribute_final_counts(eff_and_act)
eff_and_act = distribute_final_counts(eff_and_act, eff='eff_Qbb')

In [24]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.12,296,0.473997,0.550031


Now we have two more columns:
* **final_counts**: the distributed counts per volume for nexus efficiency
* **final_counts_Qbb**: the distributed counts per volume for energy efficiency

Finally we select a factor to increase the statistics (in this case x100) and compute the required simulation counts and the exposure.

In [25]:
factor = 100

#for activities in mBq to year^-1
year = (3600 * 24 * 365) / 1000

eff_and_act['final_counts_factor'] = eff_and_act.final_counts * factor
eff_and_act['final_counts_Qbb_factor'] = eff_and_act.final_counts_Qbb * factor

eff_and_act['exposure_factor'] = eff_and_act.final_counts_factor/ (eff_and_act.act * year * eff_and_act.eff) 
eff_and_act['exposure_Qbb_factor'] = eff_and_act.final_counts_Qbb_factor / (eff_and_act.act * year * eff_and_act.eff_Qbb) 

So finally we end with a DF with:
* Isotope
* G4Volume
* eff
* eff_Qbb
* act
* total_final_counts
* final_counts
* final_counts_Qbb
* **final_counts_factor**: final counts multiplied by the desired factor
* **final_counts_Qbb_factor**: same for energy efficiency
* **exposure_factor**: exposure to obtain the desired counts in nexus after the applied factor
* **exposure_Qbb_factor**: exposure to obtain the desired counts in nexus and in the energy window (2.4-2.5MeV) after the applied factor

Those last 2 colums are just a number for isotope, common to all volumes. It is an estimation from the activities and efficiencies of which exposure is needed in the whole experiment to obtain the desired counts:

In [50]:
print('For a factor = {}, we obtain the following EXPOSURES for each isotope and efficiency type:'.format(factor))
eff_and_act[['Isotope', 'exposure_factor', 'exposure_Qbb_factor']].round(12).drop_duplicates().dropna()

For a factor = 100, we obtain the following EXPOSURES for each isotope and efficiency type:


,Isotope,exposure_factor,exposure_Qbb_factor
0,214Bi,96.856168,374.643472
22,208Tl,100.057585,649.353658


In [51]:
eff_and_act

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,final_counts_factor,final_counts_Qbb_factor,exposure_factor,exposure_Qbb_factor
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.120000,296,0.473997,0.550031,47.399671,55.003139,96.856168,374.643472
1,214Bi,EP_COPPER_PLATE,3.802124e-06,1.225071e-06,82.021879,296,9.525546,11.871780,952.554555,1187.177970,96.856168,374.643472
2,214Bi,GATE_RING,3.953236e-05,1.115436e-05,1.176124,296,1.420168,1.549967,142.016781,154.996689,96.856168,374.643472
3,214Bi,HDPE_TUBE,4.443023e-05,1.370990e-05,3.070860,296,4.167470,4.974152,416.746958,497.415181,96.856168,374.643472
4,214Bi,VESSEL,1.520359e-07,5.307350e-08,3422.540000,296,15.893828,21.461054,1589.382834,2146.105399,96.856168,374.643472
5,214Bi,PEDESTAL,2.500000e-08,6.250000e-09,358.080000,296,0.273435,0.264414,27.343491,26.441425,96.856168,374.643472
6,214Bi,SHIELDING_LEAD,2.936508e-08,8.888889e-09,5321.130000,296,4.772753,5.588254,477.275278,558.825382,96.856168,374.643472
7,214Bi,ANODE_RING,4.014916e-05,9.302854e-06,1.176242,296,1.442470,1.292818,144.247041,129.281847,96.856168,374.643472
8,214Bi,SIPM_BOARD,8.291265e-05,1.696023e-05,35.635891,296,90.248981,71.407555,9024.898120,7140.755497,96.856168,374.643472
9,214Bi,OPTICAL_PAD,2.528092e-05,8.263023e-06,7.920000,296,6.115780,7.731948,611.578027,773.194796,96.856168,374.643472


We also want that after a factor, each volume has at least 12 counts (limit poisson-gauss), so we re-do the calculation of the exposure. Also, we round up the number of events (because nexus simulates by number of events), but the deciding factor will be forcing to have 12 events.

In [52]:
eff_and_act['exposure_factor_round'] = eff_and_act.final_counts_factor.apply(lambda x: int(12) if x < 12 else np.ceil(x))/ (eff_and_act.act * year * eff_and_act.eff) 
eff_and_act['exposure_Qbb_factor_round'] = eff_and_act.final_counts_Qbb_factor.apply(lambda x: int(12) if x < 12 else np.ceil(x)) / (eff_and_act.act * year * eff_and_act.eff_Qbb) 

#la needed exposure es la maxima de todas las exposures de un isotope pq es la necesaria para que el volumen q menos cuentas tiene
#llegue a 12
eff_and_act = eff_and_act.merge(eff_and_act.groupby('Isotope').exposure_factor_round.max().rename('required_exposure'), on = 'Isotope')
eff_and_act = eff_and_act.merge(eff_and_act.groupby('Isotope').exposure_Qbb_factor_round.max().rename('required_exposure_Qbb'), on = 'Isotope')


In [53]:
eff_and_act.head(1)

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,final_counts_factor,final_counts_Qbb_factor,exposure_factor,exposure_Qbb_factor,exposure_factor_round,exposure_Qbb_factor_round,required_exposure,required_exposure_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.12,296,0.473997,0.550031,47.399671,55.003139,96.856168,374.643472,98.082876,381.433405,413.401733,2067.008663


Finally we have 4 new columns:

* **exposure_factor_round**: exposure to obtain the desired counts in nexus after the applied factor but now the counts were previously forced to be at least 12 (and the rest rounded)
* **exposure_Qbb_factor_round**: same as the previous but for the energy window
* **required_exposure**: value of the maximum exposure per isotope after forcing at least 12 counts in every volume
* **required_exposure_Qbb**: same as the previous but for the energy window

In [54]:
print('For a factor = {}, integer number of events and minimum 12 counts / volume, we obtain the following EXPOSURES for each isotope and efficiency type:'.format(factor))
eff_and_act[['Isotope', 'required_exposure', 'required_exposure_Qbb']].round(12).drop_duplicates().dropna()


For a factor = 100, integer number of events and minimum 12 counts / volume, we obtain the following EXPOSURES for each isotope and efficiency type:


,Isotope,required_exposure,required_exposure_Qbb
0,214Bi,413.401733,2067.008663
22,208Tl,100.689488,654.076296


In [55]:
eff_and_act

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,final_counts_factor,final_counts_Qbb_factor,exposure_factor,exposure_Qbb_factor,exposure_factor_round,exposure_Qbb_factor_round,required_exposure,required_exposure_Qbb
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.120000,296,0.473997,0.550031,47.399671,55.003139,96.856168,374.643472,98.082876,381.433405,413.401733,2067.008663
1,214Bi,EP_COPPER_PLATE,3.802124e-06,1.225071e-06,82.021879,296,9.525546,11.871780,952.554555,1187.177970,96.856168,374.643472,96.901461,374.902884,413.401733,2067.008663
2,214Bi,GATE_RING,3.953236e-05,1.115436e-05,1.176124,296,1.420168,1.549967,142.016781,154.996689,96.856168,374.643472,97.526728,374.651475,413.401733,2067.008663
3,214Bi,HDPE_TUBE,4.443023e-05,1.370990e-05,3.070860,296,4.167470,4.974152,416.746958,497.415181,96.856168,374.643472,96.914978,375.083947,413.401733,2067.008663
4,214Bi,VESSEL,1.520359e-07,5.307350e-08,3422.540000,296,15.893828,21.461054,1589.382834,2146.105399,96.856168,374.643472,96.893778,374.799642,413.401733,2067.008663
5,214Bi,PEDESTAL,2.500000e-08,6.250000e-09,358.080000,296,0.273435,0.264414,27.343491,26.441425,96.856168,374.643472,99.181655,382.557812,413.401733,2067.008663
6,214Bi,SHIELDING_LEAD,2.936508e-08,8.888889e-09,5321.130000,296,4.772753,5.588254,477.275278,558.825382,96.856168,374.643472,97.003240,374.760538,413.401733,2067.008663
7,214Bi,ANODE_RING,4.014916e-05,9.302854e-06,1.176242,296,1.442470,1.292818,144.247041,129.281847,96.856168,374.643472,97.361751,376.724596,413.401733,2067.008663
8,214Bi,SIPM_BOARD,8.291265e-05,1.696023e-05,35.635891,296,90.248981,71.407555,9024.898120,7140.755497,96.856168,374.643472,96.857262,374.656300,413.401733,2067.008663
9,214Bi,OPTICAL_PAD,2.528092e-05,8.263023e-06,7.920000,296,6.115780,7.731948,611.578027,773.194796,96.856168,374.643472,96.922997,375.033626,413.401733,2067.008663


In [59]:
eff_and_act['exp_for_100_counts'] = 100 / (eff_and_act.act * year * eff_and_act.eff)

In [60]:
eff_and_act

,Isotope,G4Volume,eff,eff_Qbb,act,total_final_counts,final_counts,final_counts_Qbb,final_counts_factor,final_counts_Qbb_factor,exposure_factor,exposure_Qbb_factor,exposure_factor_round,exposure_Qbb_factor_round,required_exposure,required_exposure_Qbb,exp_for_100_counts
0,214Bi,DB_PLUG,1.000400e-07,3.001200e-08,155.120000,296,0.473997,0.550031,47.399671,55.003139,96.856168,374.643472,98.082876,381.433405,413.401733,2067.008663,204.339324
1,214Bi,EP_COPPER_PLATE,3.802124e-06,1.225071e-06,82.021879,296,9.525546,11.871780,952.554555,1187.177970,96.856168,374.643472,96.901461,374.902884,413.401733,2067.008663,10.168044
2,214Bi,GATE_RING,3.953236e-05,1.115436e-05,1.176124,296,1.420168,1.549967,142.016781,154.996689,96.856168,374.643472,97.526728,374.651475,413.401733,2067.008663,68.200509
3,214Bi,HDPE_TUBE,4.443023e-05,1.370990e-05,3.070860,296,4.167470,4.974152,416.746958,497.415181,96.856168,374.643472,96.914978,375.083947,413.401733,2067.008663,23.241002
4,214Bi,VESSEL,1.520359e-07,5.307350e-08,3422.540000,296,15.893828,21.461054,1589.382834,2146.105399,96.856168,374.643472,96.893778,374.799642,413.401733,2067.008663,6.093948
5,214Bi,PEDESTAL,2.500000e-08,6.250000e-09,358.080000,296,0.273435,0.264414,27.343491,26.441425,96.856168,374.643472,99.181655,382.557812,413.401733,2067.008663,354.220196
6,214Bi,SHIELDING_LEAD,2.936508e-08,8.888889e-09,5321.130000,296,4.772753,5.588254,477.275278,558.825382,96.856168,374.643472,97.003240,374.760538,413.401733,2067.008663,20.293565
7,214Bi,ANODE_RING,4.014916e-05,9.302854e-06,1.176242,296,1.442470,1.292818,144.247041,129.281847,96.856168,374.643472,97.361751,376.724596,413.401733,2067.008663,67.146035
8,214Bi,SIPM_BOARD,8.291265e-05,1.696023e-05,35.635891,296,90.248981,71.407555,9024.898120,7140.755497,96.856168,374.643472,96.857262,374.656300,413.401733,2067.008663,1.073211
9,214Bi,OPTICAL_PAD,2.528092e-05,8.263023e-06,7.920000,296,6.115780,7.731948,611.578027,773.194796,96.856168,374.643472,96.922997,375.033626,413.401733,2067.008663,15.837091
